In [1]:
from pathlib import Path
import numpy as np

processed_dir = Path("../data_processed")

X = np.load(processed_dir / "X.npy")
labels = np.load(processed_dir / "labels.npy")

print("X shape:", X.shape)
print("labels shape:", labels.shape)

X shape: (5760, 69)
labels shape: (5760,)


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

X_train, X_test, y_train, y_test = train_test_split(
    X, labels, test_size=0.2, random_state=42, stratify=labels
)

results = {}

# Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_prob = lr.predict_proba(X_test)[:, 1]
results["LR"] = {
    "accuracy": accuracy_score(y_test, lr_pred),
    "roc_auc": roc_auc_score(y_test, lr_prob),
}

# SVM (probabilities via probability=True)
svm = SVC(kernel="rbf", probability=True, random_state=42)
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_test)
svm_prob = svm.predict_proba(X_test)[:, 1]
results["SVM"] = {
    "accuracy": accuracy_score(y_test, svm_pred),
    "roc_auc": roc_auc_score(y_test, svm_prob),
}

# Neural Network (MLP)
mlp = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300, random_state=42)
mlp.fit(X_train, y_train)
mlp_pred = mlp.predict(X_test)
mlp_prob = mlp.predict_proba(X_test)[:, 1]
results["MLP"] = {
    "accuracy": accuracy_score(y_test, mlp_pred),
    "roc_auc": roc_auc_score(y_test, mlp_prob),
}

results

{'LR': {'accuracy': 0.8862847222222222, 'roc_auc': 0.42673325806909856},
 'SVM': {'accuracy': 0.8897569444444444, 'roc_auc': 0.799754768188649},
 'MLP': {'accuracy': 0.8862847222222222, 'roc_auc': 0.5765826049898692}}

In [3]:
import pandas as pd

df_results = pd.DataFrame.from_dict(results, orient="index")
df_results.index.name = "Model"
df_results

,accuracy,roc_auc
Model,,
LR,0.886285,0.426733
SVM,0.889757,0.799755
MLP,0.886285,0.576583


In [4]:
export_dir = Path("../reports")
export_dir.mkdir(exist_ok=True)

csv_path = export_dir / "patient01_model_metrics.csv"
df_results.to_csv(csv_path, index=True)

print("Saved metrics to:", csv_path)

Saved metrics to: ../reports/patient01_model_metrics.csv


In [5]:
# Example structure – adapt names if needed
forecasting_results = {
    "LR":  {"SS": ss_lr,  "FPR_h": fpr_lr,  "TiW": tiw_lr,  "Brier": bs_lr,  "Brier_SS": bss_lr},
    "SVM": {"SS": ss_svm, "FPR_h": fpr_svm, "TiW": tiw_svm, "Brier": bs_svm, "Brier_SS": bss_svm},
    "MLP": {"SS": ss_mlp, "FPR_h": fpr_mlp, "TiW": tiw_mlp, "Brier": bs_mlp, "Brier_SS": bss_mlp},
}

df_forecasting = pd.DataFrame.from_dict(forecasting_results, orient="index")
df_forecasting.index.name = "Model"

csv_path_fore = export_dir / "patient01_forecasting_metrics.csv"
df_forecasting.to_csv(csv_path_fore, index=True)

print("Saved forecasting metrics to:", csv_path_fore)

NameError: name 'ss_lr' is not defined